### 필요한 모듈 import 하기

In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [10]:
#Mecab 형태소 분석기
from konlpy.tag import Mecab
mecab = Mecab(dicpath='C:\MeCab\dic\mecab-ko-dic') #dicpath는 컴퓨터 환경 마다 경로가 다를 수 있다

In [11]:
def mecab_noun(text): #단어
    mecab_list = mecab.nouns(text)
    return mecab_list

In [12]:
def mecab_morph(text): #형태소
    mecab_list = mecab.morphs(text)
    return mecab_list

## 1. 트레이닝 데이터 불러오기

In [13]:
file_path = 'H:/M/python/supervision/MJ_2019/utf8/'
san_train_file_path = 'H:/M/python/supervision/MJ_2019/'
file_name = "sanjae_yn_190727_train_set.txt"

In [14]:
df_train = pd.read_table(san_train_file_path+file_name, encoding='utf-8')

In [15]:
df_train = df_train.drop("Unnamed: 0",axis=1)

In [16]:
df_train[df_train.sanjae_yn.isnull()]

,sanjae_yn,num,id,field_1,title,content


### 데이터 전처리

In [17]:
#메캅 형태소 분석 (단어 or 형태소 분석 버전)
df_train["mecab_content"] = df_train["content"].apply(lambda x : mecab_morph(x))

In [18]:
#메캅 품사 분석 (명사 + 형용사 + 부사 등등)
df_train["mecab_content2"] = df_train["content"].apply(lambda x : mecab.pos(x))

In [19]:
df_train["mecab_content2"]

0        [(1, SN), (월, NNBC), (중순, NNG), (쯤, XSN), (에, ...
1        [(◆, SY), (답, NNG), (담, XR), (한, XSA+ETM), (맘,...
2        [(안녕, NNG), (하, XSV), (세요, EP+EF), (저, NP), (는...
3        [(제, NP), (가, JKS), (얼마, NNG), (전, NNG), (에, J...
4        [(16, SN), (년, NNBC), (도, JX), (8, SN), (월, NN...
5        [(저희, NP), (아버지, NNG), (가, JKS), (하, VV), (시, ...
6        [(제, NP), (가, JKS), (직장, NNG), (을, JKO), (2013...
7        [(먼저, MAG), (,, SC), (근로, NNG), (기준, NNG), (법,...
8        [(작, VA), (은, ETM), (동네, NNG), (카페, NNG), (에서,...
9        [(2, SN), (년, NNBC), (만근, NNG), (기준, NNG), (연차...
10       [(저, NP), (는, JX), (관리자, NNG), (입니다, VCP+EF), ...
11       [(저, NP), (는, JX), (2014, SN), (년, NNBC), (6, ...
12       [(조그만, VA+ETM), (소기업, NNG), (의, JKG), (과장, NNG...
13       [(?, SY), (안녕, NNG), (하, XSV), (세요, EP+EF), (다...
14       [(?, SY), (안녕, NNG), (하, XSV), (십니까, EP+EF), (...
15       [(?, SY), (현재, NNG), (직장, NNG), (에서, JKB), (근무...
16       [(?, SY), (질문, NNG), (이, JKS), (있, VA), (습니다, .

In [20]:
def apply_pos(content_pos):    
    pos_list = []
    pos_tagging = ['NNG','NNP','MAG','XR','SL'] #명사 부사 어근 외국어
    for x in content_pos:#NNG,NNP,VA+ETM,MAG,NNBC
        if x[1] in pos_tagging:
            pos_list.append(x[0])
    return pos_list

In [21]:
df_train["mecab_pos"] = df_train["mecab_content2"].apply(lambda x : apply_pos(x))

In [22]:
df_train

,sanjae_yn,num,id,field_1,title,content,mecab_content,mecab_content2,mecab_pos
0,O,280.0,382_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,2월초에 회사일을 하다가 허리를 다쳐서 추간판탈골증 진단을 받았습니다,1월중순쯤에 회사일을 하다가 허리를 다쳐서 추간판탈골증 진단을 받았습니다 사장이 처...,"[1, 월, 중순, 쯤, 에, 회사, 일, 을, 하, 다가, 허리, 를, 다쳐서, ...","[(1, SN), (월, NNBC), (중순, NNG), (쯤, XSN), (에, ...","[중순, 회사, 일, 허리, 추간판, 골증, 진단, 사장, 처음, 산재, 처리, 답..."
1,O,2689.0,3822_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,"근로자의 권리(퇴직금,산재,실업급여)를 찿을수 있을까요??",◆ 답담한 맘에 질문드리오니..?명쾌한 답변 부탁드립니다 !! 간절합니다 !!??근...,"[◆, 답, 담, 한, 맘, 에, 질문, 드리, 오니, ., .?, 명쾌, 한, 답...","[(◆, SY), (답, NNG), (담, XR), (한, XSA+ETM), (맘,...","[답, 담, 맘, 질문, 명쾌, 답변, 부탁, 간절, 근속, 연수, 보험, 가입, ..."
2,O,7577.0,11166_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,산재재요양신청,안녕하세요 저는 2014년6월에 회사에서 사고를 당했습니다. 사고로 인해 우측 손가...,"[안녕, 하, 세요, 저, 는, 2014, 년, 6, 월, 에, 회사, 에서, 사고...","[(안녕, NNG), (하, XSV), (세요, EP+EF), (저, NP), (는...","[안녕, 회사, 사고, 사고, 우측, 손가락, 절단, 회사, 산재, 처리, 산재, ..."
3,O,8235.0,11845_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,일용직 알바를 하다가 다쳤습니다..,제가 얼마전에 건설현장에서 무거운물건을 옮기는작업을하다가 굵은철근이 다리에걸려 넘어...,"[제, 가, 얼마, 전, 에, 건설, 현장, 에서, 무거운, 물건, 을, 옮기, 는...","[(제, NP), (가, JKS), (얼마, NNG), (전, NNG), (에, J...","[얼마, 전, 건설, 현장, 물건, 작업, 철근, 다리, 철근, 다리, 너무, 피,..."
4,X,10651.0,14492_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,이혼후 재산처분이,16년도8월에 이혼을 했습니다. 그때 위자료대신 지금가지고있는 집 담보대출에 대한 ...,"[16, 년, 도, 8, 월, 에, 이혼, 을, 했, 습니다, ., 그때, 위자료,...","[(16, SN), (년, NNBC), (도, JX), (8, SN), (월, NN...","[이혼, 그때, 위자료, 대신, 지금, 집, 담보, 대출, 조건, 합의, 문제, 담..."
5,O,12622.0,16708_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,조경공사 현장에서 일하던 일용직 일꾼이다쳤습니다,저희아버지가하시는 조경공사 현장에서 일하던 일용직 일꾼이다쳤습니다저희현장은 공사금액...,"[저희, 아버지, 가, 하, 시, 는, 조경, 공사, 현장, 에서, 일, 하, 던,...","[(저희, NP), (아버지, NNG), (가, JKS), (하, VV), (시, ...","[아버지, 조경, 공사, 현장, 일, 일용직, 일, 현장, 공사, 금액, 현장, 산..."
6,X,12974.0,17127_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,연차수당 청구 가능한 사업장 맞나요?,제가 직장을 2013년 3월 27일 입사해서 2016년 3월 31일까지 일하고 퇴사...,"[제, 가, 직장, 을, 2013, 년, 3, 월, 27, 일, 입사, 해서, 20...","[(제, NP), (가, JKS), (직장, NNG), (을, JKO), (2013...","[직장, 입사, 일, 퇴사, 연차, 사용, 그냥, 안, 연차, 유급, 휴가, 말, ..."
7,X,15807.0,20153_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,근로법과 근로계약서 상의 휴가에 대한 질문입니다,"먼저, 근로기준법 제60조에 의하면 내용이 이렇게 써 있습니다.?제60조(연차 유급...","[먼저, ,, 근로, 기준, 법, 제, 60, 조, 에, 의하, 면, 내용, 이, ...","[(먼저, MAG), (,, SC), (근로, NNG), (기준, NNG), (법,...","[먼저, 근로, 기준, 법, 조, 내용, 이렇게, 조, 연차, 유급, 휴가, 사용,..."
8,O,16517.0,21036_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,카페알바 2도화상 산재보험..,"작은 동네 카페에서(5인미만,프랜차이즈x) 아르바이트를 하고있는 학생입니다알바스케줄...","[작, 은, 동네, 카페, 에서, (, 5, 인, 미만, ,, 프랜차이즈, x, )...","[(작, VA), (은, ETM), (동네, NNG), (카페, NNG), (에서,...","[동네, 카페, 인, 미만, 프랜차이즈, x, 아르바이트, 학생, 알바, 스케줄, ..."
9,X,16642.0,21267_jikeopbyoeng_eopmusangjilbyeong_id,근로기준,계약직 연차 보상관련,2년만근기준 연차휴가 30개 발생입니다.2013.6.17~2014.6.16 1년 계...,"[2, 년, 만근, 기준, 연차, 휴가, 30, 개, 발생, 입니다, ., 2013...","[(2, SN), (년, NNBC), (만근, NNG), (기준, NNG), (연차...","[만근, 기준, 연차, 휴가, 발생, 계약, 이후, 재계약, 연장, 합의서, 작성,..."


In [23]:
df_train['sanjae_yn'] = df_train['sanjae_yn'].map({'O': 1, 'X': 0, 'o' : 1, 'x':0}) # ox 통일

In [24]:
train_labels = df_train["sanjae_yn"]

In [25]:
train_doc = df_train["mecab_content"]   

In [26]:
train_doc

0        [1, 월, 중순, 쯤, 에, 회사, 일, 을, 하, 다가, 허리, 를, 다쳐서, ...
1        [◆, 답, 담, 한, 맘, 에, 질문, 드리, 오니, ., .?, 명쾌, 한, 답...
2        [안녕, 하, 세요, 저, 는, 2014, 년, 6, 월, 에, 회사, 에서, 사고...
3        [제, 가, 얼마, 전, 에, 건설, 현장, 에서, 무거운, 물건, 을, 옮기, 는...
4        [16, 년, 도, 8, 월, 에, 이혼, 을, 했, 습니다, ., 그때, 위자료,...
5        [저희, 아버지, 가, 하, 시, 는, 조경, 공사, 현장, 에서, 일, 하, 던,...
6        [제, 가, 직장, 을, 2013, 년, 3, 월, 27, 일, 입사, 해서, 20...
7        [먼저, ,, 근로, 기준, 법, 제, 60, 조, 에, 의하, 면, 내용, 이, ...
8        [작, 은, 동네, 카페, 에서, (, 5, 인, 미만, ,, 프랜차이즈, x, )...
9        [2, 년, 만근, 기준, 연차, 휴가, 30, 개, 발생, 입니다, ., 2013...
10       [저, 는, 관리자, 입니다, 저희, 업소, 주방, 에서, 일, 하, 던, 사람, ...
11       [저, 는, 2014, 년, 6, 월, 9, 일, 입사, 하, 여, 개인, 적, 인...
12       [조그만, 소기업, 의, 과장, 입니다, ., 다름, 아니, 고, ., 2013, ...
13       [?, 안녕, 하, 세요, 다름, 이, 아니, 라, 제, 가, 공장, 에서, 고정,...
14       [?, 안녕, 하, 십니까, 노, 무사, 님, 아무것, 도, 모르, 고, 사업, 은...
15       [?, 현재, 직장, 에서, 근무, 한지, 13, 개월, 되, 었, 습니다, ., ...
16       [?, 질문, 이, 있, 습니다, ., ?, 제, 가, 근무, 하, 는, 곳, 이,.

In [27]:
train_doc2 = df_train["mecab_pos"]
train_doc2

0        [중순, 회사, 일, 허리, 추간판, 골증, 진단, 사장, 처음, 산재, 처리, 답...
1        [답, 담, 맘, 질문, 명쾌, 답변, 부탁, 간절, 근속, 연수, 보험, 가입, ...
2        [안녕, 회사, 사고, 사고, 우측, 손가락, 절단, 회사, 산재, 처리, 산재, ...
3        [얼마, 전, 건설, 현장, 물건, 작업, 철근, 다리, 철근, 다리, 너무, 피,...
4        [이혼, 그때, 위자료, 대신, 지금, 집, 담보, 대출, 조건, 합의, 문제, 담...
5        [아버지, 조경, 공사, 현장, 일, 일용직, 일, 현장, 공사, 금액, 현장, 산...
6        [직장, 입사, 일, 퇴사, 연차, 사용, 그냥, 안, 연차, 유급, 휴가, 말, ...
7        [먼저, 근로, 기준, 법, 조, 내용, 이렇게, 조, 연차, 유급, 휴가, 사용,...
8        [동네, 카페, 인, 미만, 프랜차이즈, x, 아르바이트, 학생, 알바, 스케줄, ...
9        [만근, 기준, 연차, 휴가, 발생, 계약, 이후, 재계약, 연장, 합의서, 작성,...
10       [관리자, 업소, 주방, 일, 사람, 근무, 시간, 핸드폰, 오락, 튀김, 기, 등...
11       [입사, 개인, 사정, 퇴사, 간, 근무, 유급, 휴가, 사용, 상기, 같이, 안,...
12       [소기업, 과장, 회사, 공장장, 근무, 여름, 속, 안, 병가, 조퇴, 내지, 연...
13       [안녕, 공장, 고정, 알바, 일, 전, 기계, 오작동, 머리, 두피, 동전, 사고...
14       [안녕, 노, 무사, 아무것, 사업, 운영, 제대로, 바람, 잘, 날, 없이, 운영...
15       [현재, 직장, 근무, 날, 퇴사, 의사, 회사, 통보, 회사, 근무, 말, 쿨, ...
16       [질문, 근무, 곳, 인터넷, 언론사, 구성원, 사업자, 포함, 보험, 해당자, 발.

In [28]:
def list2str(text):
    words = " ".join(text)
    return words

In [29]:
#train_doc = train_doc.apply(lambda x : list2str(x)) #리스트에서 텍스트형태로 변환 (형태소 분석을 할 경우 주석을 해제)
train_doc2 = train_doc2.apply(lambda x : list2str(x)) #리스트에서 텍스트형태로 변환

In [30]:
def remove_trash(text):
    ret = re.sub(r'[-=+,\t\"\|\\#/\?◆:;^.@*\""~&%!\'|\(\)\[\]\<\>`♡}{「」【】·\'...]', '', text)
    return ret

In [54]:
train_doc = train_doc.apply(lambda x:remove_trash(x))

In [31]:
train_doc2 = train_doc2.apply(lambda x:remove_trash(x))

### 데이터 분석 (랜덤 포레스트 기계 학습)

In [32]:
vectorizer = CountVectorizer(tokenizer=str.split,max_features = 18000,max_df=30,min_df=2,ngram_range=(3,7)) #최대값 18000 dtm 생성
#train_doc_term_mat = vectorizer.fit_transform(train_doc)
train_doc_term_mat2 = vectorizer.fit_transform(train_doc2)

In [33]:
clf = RandomForestClassifier(n_estimators=1000,max_depth=50, random_state=0)
clf.fit(train_doc_term_mat2, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [34]:
clf.score(train_doc_term_mat2,train_labels) 

0.6832747041893188

## 2.테스트 파일 불러오기

In [35]:
import json
text_path = 'C:/Users/KHJ/TextMining/sanjae/'
daum_file = 'daum2.txt'
naver_file = 'naver.txt'

### JSON 파일을 DataFrame 형태로 변환 (전처리 과정은 문서 파일마다 달라질 수 있음)

In [36]:
def yield_line_json_daum(path):
    with open(path, 'r',encoding='utf-8-sig') as f:
        for line in f:
            line = json.dumps(line)
            str_data = json.loads(line)
            data = str_data.split(":")
            id_val = data[1].split(",")
            ex_content = str_data.split('""content"":') #content부터
            ex_content2 = ex_content[1].split('""writer""') #writer까지 문장 split
            content = ex_content2[0]
            yield id_val[0],content

In [37]:
def yield_line_json_naver(path):
    with open(path, 'r',encoding='utf-8-sig') as f:
        for line in f:
            line = json.dumps(line)
            str_data = json.loads(line)
            data = str_data.split(":")
            id_val = data[1].split(",")
            ex_content = str_data.split('""content"":') #sentences부터
            ex_content2 = ex_content[1].split('""com"":') #com까지 문장 split
            content = ex_content2[0]
            yield id_val[0],content

In [38]:
daum_data = yield_line_json_daum(text_path+daum_file)
naver_data = yield_line_json_naver(text_path+naver_file)

In [39]:
daum_df = pd.DataFrame.from_records(list(daum_data), columns = ['id','content'])
naver_df = pd.DataFrame.from_records(list(naver_data), columns = ['id','content'])

In [30]:
daum_df

,id,content
0,"""""daum_cafe_1xxxx1_1""""""","""""제1조(목적) 이 법은 「고용보험 및 산업재해보상보험의 보험료징수 등에 관한 ..."
1,"""""daum_cafe_1xxxx1_2""""""","""""제1장 총칙 제1조(목적) 이 법은 고용보험과 산업재해보상보험의 보험관계의 ..."
2,"""""daum_cafe_1xxxx1_3""""""","""""제1장 총칙<개정 2009.12.30> 제1조 (목적) 이 법은 고용보험과..."
3,"""""daum_cafe_1xxxx1_4""""""","""""고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행규칙 법령 제..."
4,"""""daum_cafe_1xxxx1_5""""""","""""대법원 2010.8.19. 선고 2010두5141 판결 【추가상병불승인처분취소..."
5,"""""daum_cafe_1xxxx1_6""""""","""""1.산재로 보상되지 않은 부분이라면 어떤 경우인지요? 현재 회사를 그만 둔 상..."
6,"""""daum_cafe_1xxxx1_7""""""","""""법무법인 조율의 정진 변호사 입니다. 1. 산재로 보상받지 못한 부분이..."
7,"""""daum_cafe_1xxxx1_8""""""","""""우선 네이버에 답변 감사합니다. 추가 질문하나를 더 드릴려고 합니다. 임의가입..."
8,"""""daum_cafe_1xxxx1_9""""""","""""법무법인 조율의 정진 변호사 입니다. 임의가입이라면 사고 후에 임의가입은 ..."
9,"""""daum_cafe_1xxxx1_10""""""","""""네이버를 통해 카페를 알게 되었어요. 사건개요는 아래와 같이 읽어보셨으리라 믿..."


In [66]:
#메캅 형태소 분석 (다음)
daum_df["mecab_morph"] = daum_df["content"].apply(lambda x : mecab_morph(x))
daum_df["mod_mecab_morph"] = daum_df["mecab_morph"].apply(lambda x: list2str(x))

In [67]:
#메캅 형태소 분석 (네이버)
naver_df["mecab_morph"] = naver_df["content"].apply(lambda x : mecab_morph(x))
naver_df["mod_mecab_morph"] = naver_df["mecab_morph"].apply(lambda x: list2str(x))

In [40]:
#메캅 품사 추출 (다음)
daum_df["mecab_pos"] = daum_df["content"].apply(lambda x : mecab.pos(x))
daum_df["mecab_pos2"] = daum_df["mecab_pos"].apply(lambda x : apply_pos(x))
daum_df["mod_mecab_pos"] = daum_df["mecab_pos2"].apply(lambda x: list2str(x))

In [41]:
#메캅 품사 추출 (네이버)
naver_df["mecab_pos"] = naver_df["content"].apply(lambda x : mecab.pos(x))
naver_df["mecab_pos2"] = naver_df["mecab_pos"].apply(lambda x : apply_pos(x))
naver_df["mod_mecab_pos"] = naver_df["mecab_pos2"].apply(lambda x: list2str(x))

In [68]:
test_daum_doc = daum_df["mod_mecab_morph"]
test_naver_doc = naver_df["mod_mecab_morph"]

In [42]:
test_daum_doc = daum_df["mod_mecab_pos"]
test_naver_doc = naver_df["mod_mecab_pos"]

In [43]:
test_daum_doc = test_daum_doc.apply(lambda x:remove_trash(x))
test_naver_doc = test_naver_doc.apply(lambda x:remove_trash(x))

### 테스트 파일 벡터화 (변환)

In [44]:
test_daum_doc_term_mat = vectorizer.transform(test_daum_doc)
test_naver_doc_term_mat = vectorizer.transform(test_naver_doc)

In [45]:
test_daum_doc

0        조 목적 법 고용 보험 산업재 보상 보험 보험료 징수 법률 보험 관계 성립 신고 고...
1        장 총칙 조 목적 법 고용 보험 산업재 보상 보험 보험 관계 성립 소멸 보험료 납부...
2        장 총칙 개정 조 목적 법 고용 보험 산업재 보상 보험 보험 관계 성립 소멸 보험료...
3        고용 보험 산업재 보상 보험 보험료 징수 법률 시행 규칙 법령 제공 법제처 XML ...
4        법원 선고 판결 추가 상병 불승인 처분 취소 공 판시 사항 산업재 보상 보험법 요양...
5        산재 보상 부분 경우 현재 회사 그만 상태 퇴직금 각종 세금 만약 법 절차 행 예상...
6        법무법인 조율 정진 변호사 산재 보상 부분 손해 배상액 산재 보상 부분 말 예 피해...
7        우선 네이버 답변 감사 추가 질문 더 임의 가입 대상 사고 후 가입 가능 가능 보상...
8        법무법인 조율 정진 변호사 가입 사고 후 임의 가입 안 산재 처리 산재 처리 손해 ...
9        네이버 카페 사건 개요 아래 같이 이제 시간 산재 보험 청구 부모 연금 산재 청구 ...
10       법무법인 조율 정진 변호사 일 가슴 사건 단순 언제 시간 사무실 상담 우선 회사 위...
11       안녕 네이버 지식인 답변 감사 오전 사건 발생 재해자 나이 월급 산재 판정 부위 발...
12       법무법인 조율 정진 변호사 어제 전화 상담 대략 답변 장애 급 상태 질문 내용 구체...
13       작년 구미 소재 휴대폰 생산 공장 일 원인 불상 사고 화상 산재 처리 산재 등급 급...
14       법무법인 조율 정진 변호사 금액 얼마 나중 신체 장애 치료비 내역 손해 배상 얼마 ...
15       변호사 안녕 일용 근로자 건축 공사장 일 리프트 바퀴 오른쪽 손가락 사고 산재 승인...
16       안녕 경기도 편의점 운영 너무나 억울 머리 일 이렇게 글 사건 편의점 주말 알바 토.

### 테스트 파일 예측

In [46]:
y_daum = clf.predict(test_daum_doc_term_mat) #결과 예측하기
y_naver = clf.predict(test_naver_doc_term_mat) #결과 예측하기

In [47]:
daum_df["result"] = y_daum
naver_df["result"] = y_naver

In [48]:
daum_view_df = daum_df[["content","result"]]
naver_view_df = naver_df[["content","result"]]

In [49]:
daum_view_df[daum_view_df.result == 1].shape 

(705, 2)

In [50]:
naver_view_df[naver_view_df.result == 1].shape 

(3705, 2)

In [51]:
pd.set_option('display.max_colwidth', 500)

### 테스트 파일 화면에서 미리보기

In [52]:
import random
daum_view_df[daum_view_df.result == 1].sample(n=100, random_state=random.randint(0, 77))

,content,result
12349,"""""답답한 마음에 인터넷 검색을 하다 카페를 보고 문의하게 되었습니다. 아빠가 지난4월 9일회사에 출근 하시자마자 급성심근경색으로 돌아가셨습니다.근무지인 경비실에 한시간정도 쓰러진채 나중에 발견되어 부검까지 하고야 말았습니다. 회사에 입사한지는 일주일도 채 되지 않아 다들 산재가 어렵다고 합니다. 과로를 입증하라 하지만 .... 그게 쉬운일이 아닌듯 합니다. 이틀은 일용직으로 일해보고 3일부터 근로계약서를 작성하고 5일째 되던 새벽에 돌아가셨습니다. 새벽 6시 30분 야근조와 교대하시고 4시30분까지 일하시는 컨테이너 임대회사 경비일을 하셨습니다. 처음 적응하시느라 잘보이시려고 열심히 일하신듯 합니다. 출근은 집에서 새벽 5시쯤 나가셨고 6시쯤 도착하셔서 일을 하신듯 합니다. 제가 궁금한건 이 회사 다니시기 전 3월 9일까지 화력발전소 경비일을 하셨는데 거기서 일이 아주 힘드셨다고 했습니다. 누적된 과로가 아무래도 영향이 있었던거 같은데 중간에 20일 정도 쉬신것 때문에 이...",1
18456,"""""산재 재해자 확인서 질문요 비공개 질문 427건 질문마감률96.1% 2015.01.09. 18:18 재해자 확인서에 일일 임금을 적으라고 되어 있습니다.그럼 기본급을 일요일 빼고 26일 나누기 하면 되나요? 그리고 이 서류 넣으면 산재 승인은 언제쯤 나나요? 산재지기(taurusbj)님의 답변입니다. 채택답변수150 2015.01.10. 12:10 질문자 인사궁금한 점이 시원하게 해결되었어요! 산재 재해자 확인서 질문 산재119 에서 답변드리겠습니다.1. 재해자님의 산재사고 관련 공단에서 재해자확인서를 요청하셨나 봅니다. 실무에서 보면 각각의 공단지사 마다 재해자확인서 양식이 약간씩 차이는 있습니다. 즉"", 재해자님사건을 위한 재해자확인서가 아닌 일반적인 재해자확인서로 되어 있...",1
18409,"""""산업재해시 보상정도 naj**** 질문 4건 질문마감률100% 2010.05.11. 21:51 고소작업차량에 탑승후 작업자의 오조작으로 인하여 버켓이 전주와 부딪히면서 작업자의 오른쪽 팔꿈치 복합골절 및 오른쪽 어깨 관절 탈골에 상해가 발생 했습니다. 진단 : 6개월 작업자는 일용직 근로자로 일당13만원에 계약되있고 최근 3개월 평균 임금은 270만원 입니다. 산재처리를 하면 사업주는 얼마나 부담을 해야되며 산재자는 얼마나 보상받을수 있나요..?산재지기(taurusbj)님의 답변입니다. 채택답변수100 2010.05.11. 22:42 질문자 인사답변 내용이 도움 많이 되었습니다~ 답변드리겠습니다. 산재보험 가입사업장 이면서 근로자 30인 미만의 사업장이라면 산재처리와 관련해서 추가적 경제적 부담...",1
8621,"""""안녕하십니까~저는 현재 모 업체에서 안전관리자로 재직중입니다.. 제가 입사한지는 1개월가량 지났는데요~~ 입사하기 전에 산업재해가 2건이 터져서 제가 안전관리자로 고용이 되었습니다.. 현재 한분은 완치되어서 현장에서 작업을 하고계신데요 다른한분은 현재 병원에서 생활하고계십니다.. 그분의 사건개요를 말하고자 하면은 높은 1.5~1M가량의 사다리에서 작업하다가 떨어지셨습니다.. 떨어지면서 어깨를 다치셔서 현재 1차 수술은 끝이났지만~ 그분 말씀으로는 2차 수술을 해야한다라고 하셨습니다.. 그 와중에 머리가 아프다 ~~어디가 아프다 라고하셔서 현재 입원생활을 유지하고계십니다.. 회사에서는 근무중에 다친 허리부분에 대한 산재처리를 하였지만 그분께서 머리와 다른곳이 아프다고 계속 산재를 연장하려고 하십니다.. 회사입장에서는 현장상황상 한사람을 계속 입원치료를 시켰을경우 작업 공백도 크고 또 예전에 앓았던 지병까지 부담하기에는 골치가 아프다라고 하시네요"",,, 궁금한점은 회...",1
18541,"""""산업재해관련 비공개 질문 178건 질문마감률98.8% 2015.11.23. 17:43 뇌진탕과 허리골절 로 산재승인받아 종합병원에서 치료받다가 머리는 좋아져서 거주지로 전원하여 정형외과에서 통원치료중입니다1)머리가 다시아파요 처음병원에서 신경과 치료가능하나요?2공단에 승인 받나요?ㅡ최초산재승인처럼 절차를밟아야하나요?3지금 치료병원에 이야기해야하나요"",4 절차는요?두서없네요 아시는데로 답변부탁합니다 (taurusbj)님의 답변입니다. 채택답변수237 2015.11.23. 20:52 질문자 인사답변내용이 많은 도움 되었습니다. 산업재해관련 산재119 에서 답변드리겠습니다. 1. 종합병원 신경외과 치료는 언제까지 받으셨나요? 최초 병원으로 미리 공단에 병행진료신청을 하신...",1
12616,"""""2006. 2. 7일 사고로 입원 손가락 좌측 3"", 4번 수지 원지골 개방성 골절 및 3,""4수지 말단부 압궤상으로 4월 21일 퇴원후 현재통원 치료중(4개월됨) 산재승인(6월 30일)- 연장 or 종결 처리결정. 회사에서 산재처리만 해주면 끝이라며 더이상은 금전적인 위로금은 인정할수 없다고 주장.(무노동 무임금원칙 주장) 회사 노조는 뒷짐.(어영 노조임)- 도움을 받을수 없습니다. 회사에 근재관련 문의 - 동부화재에 50명 인원으로 가입 혜택 산재 9급이상의 중증 상태만 해당 된다고 말씀하십니다. 혜택의 대상이 안된다고 하십니다. 회사는 원칙에 준하여서만 얘기 하십니다. 종결이 얼마남지 않아서 후휴 증상도 남을것 같은데 산재 보상(종결후)외 회사 보상은 받을수 없는지 있다면 방법이 무엇인지 간곡히 좋은 방법 있으면 부탁 드립니다. 수고하십시요."""""",""",1
10236,"""""헉..넘 크게 다치셨습니다..정말 힘내시고 쾌차하시길 빌고 또 빕니다.... 일단 업무상 재해로 판명되시면 요양급여"",휴업급여,장해급여,""간병급여등의 혜택을 보실 수 있을 거 같습니다... 힘 내세요...."""""",""",1
15431,""""" 산재나 보험사나 둘중 하나를 선택합니다 중복보상은 안됩니다 산재로 처리할시에는 배상받지 못한 위자료는 자동차 보험사를 상대로 청구 할 수 있습니다 차대차 사고인 경우에는 자차 자손에는 해당되지 않습니다 동승자인 경우 호의동승과실 20 입니다 상대차량 보험사에서 배상후 피해차량 과실부분 만큼 피해차량 보험사에 구상하게 됩니다 과실에 대한부분은 일정부분 각오하셔야 하며 사고상황에 따라서 변수가 무척이나 많으나 10 20 정도로 사료됩니다 참고가 되셨다면 좋겠습니다 """""",""",1
18513,"""""산재가 가능한가요 비공개 질문 8건 질문마감률100% 2015.10.19. 13:02 왼쪽무릅연골이 다쳐 산재로 수술후 재활이 안된상태로 지금껏 생활하다가다리쪽이 아파서 병원에서 확인한 결과 4번 디스크라고 하네요이럴경우 디스크 수술도 산재로 가능한가요다리가 불편해 힘든일 무거운 물건을 들지도 않고 쩔둑거리며 지금까지 걸어왔는데이제 허리까지 수술을 하라고 하니 깜깜하네요잘못된 자세로 허리까지 영향이 왔는데 산재 처리가 안되면 수술비도 비싸서 걱정이네요산재지기(taurusbj)님의 답변입니다. 채택답변수212 2015.10.19. 16:24 질문자 인사답변 내용이 많은 도움 되었습니다. 산재가능여부 산재119에서 답변드리겠습니다.1. 산업재해는 신청상병에 대해 승인심사를 하는 행정청...",1
18426,"""""산재신청중 퇴사하면 산재처리가 안되나요? 비공개 질문 21건 질문마감률85.7% 2010.08.14. 13:44 디스크탈출증으로 판명이 나서 수술을 했고 산재신청을 했는데 질병위원회에 의뢰를 했다고문자를 받았습니다.문제는 제가 그일을 계속 할수가 없는 몸 상태라서 퇴직을 할려 하는데요.얘기듣기론 산재신청중에 퇴직을 하면 산재가 무산된다고 합니다.그게맞는 얘긴지 ....또한가진 제가 퇴사를 하면 실업급여를 받을수 있을지 그것도 의문 입니다.현재는 병원 처방약을 계속 먹고 있는 상태이구요.질문은 퇴직을 하면 산재 승인이 무산될까요?불승인이나서 퇴직을 해도 실업급여를 받을수 있는지요?좋은답변기다립니다. 산재지기(taurusbj)님의 답변입니다. 채택답변수120 2010.08.15. 00:42 질문자 인사빠른 ...",1


In [53]:
naver_view_df[naver_view_df.result == 1].sample(n=100, random_state=random.randint(0, 77))

,content,result
181105,"""""저번주에 상담글을 올렸는데요. 신랑이 오늘 회사가서 산재처리 해달라고 이야기 햇더니 안해준다고 공상처리를 해준다고 했다는데요 공상처리를 해준다는게 어떤건지 궁금해서요. 신랑은 퇴사를 할려고 했었고 당장 일도 못하는데 출근하는것도 그렇고해서 퇴사를 말일자로 한다고 했데요. 혹시 퇴사를 하고 치료 3-4달 받고 완치하고 난 뒤에 산재처리 가능한가요? 저 입장 같아선 당장 산재신고하고 싶은데. 밀린 월급도 3달치 받아야하고 "","" 퇴사하면 퇴직금도 받아야하는데 산재신고하게되면 회사에서 나몰라라하고 월급이고 퇴직금이고 안 줄것 같거든요."""""",""",1
135486,"""""산재보험이란 업무상 재해에 대한 보상 이므로 즉 야근 하신 경우 사업주의 구체적이 지시가 있었는가가 중요하겠구요 사업주의 지시에 의해 근무중에 이와 같은 사고를 당하신 경우라면 기타사고에 의하 재해라고 볼 수 있습니다 산재법상 제38조에 의거 타인의 폭력행위에 의하여 근로자가 사상한 경우 다음각호의 요건에 해당되는 경우에는 이를 법 제 54조의 규정에 의한 제3자의 행위에 의한 업무상 재해로 볼 수 있습니다 1 재해발생경위 및 사상한 근로자가 담당한 업무의 성질이 가해행위를 유발할 수 있다고 사회통념상 인정 될 것 2 타인의 가해행위와 사상한 근로자의 사상간에 상당인과관계가 있을것 위와 같이 설명하고 있기 때문에 산재처리가 가능 할 것으로 생각됩니다 그다음은 산재처리가 된다면 그 이후의 절차에 관해서는 우선 미가입산재로 처리가 되기 때문에산재보험에 가입하고 있지는 않 았으나 법에 의하여 의무적으로 가입하였어야만 하는 사업장에서 ...",1
146358,"""""안녕하십니까 건설현장에 근무하는 안전관리자입니다몇 일전 일요일 현장에서 사고가 발생했는데"", 산재가 가능한지 궁금합니다 공사과장이 일용직 근로자 5명을 용역회사에 신청하여 현장에서 작업지시를 하고 일을 시켰답니다 작업내용은 지하실에 있는 쓰레기를 폐기물 압롤박스 앞까지 운반하여 정돈만 하고 상차는 하지 않도록하고, 옆에 적재만 해 두라고 용역팀장에게 공사과장이 작업지시를 직접 전달하고, 용역팀장은 피재자외 1명에게 공사과장이 지시한 내용데로 폐기물을 압롤박스에 상차하지 않고 옆에 적재하라고 지시를 하였는 데,"" 피재자가 임의로 폐기물 압롤박스에 올라갔다가 실족하여 추락한 사고입니다 이럴 경우 산재처리가 가능한지요 """""",""",1
135500,"""""안녕하세요 우선 이런 좋은 카페가 있어서 너무 좋습니다 다른게 아니라 저희 아버지께서 일하시다 눈을 다치셨습니다 2007년 8월 23일 눈을 다치셔서 수술은 했습니다 산재 처리를 할려고 하다보니 어떻게 해야하는지 잘 모르겠어서 이렇게 글을 남깁니다 우선 퇴원은 내일 8월 30일 해야하는데 병원비도 내야하고 어떻게 하는지 좀 알려주세요 """""",""",1
87649,"""""감기조심하세요 박사님 일단 궁굼한 사항을 질문드립니다 A현장 협력업체 근로자가 있습니다 이 근로자는 인근에 위취한 같은 없체의 B현장의 일손이 필요하다는 연락을 받고 작업반장과 함께 다음날 B현장에서 일을 하러 갔습니다 일을 하던 도중 산재사고가 발생 하였고 현재 입원 중입니다 질문1 A현장에서 일을하던 도중 B현장으로 잠깐 일을 도와주러 간 것이기 때문에 출장으로 봐도 되나요 질문2 일용직은 출장으로 볼수 없다면 B현장의 산재보험으로 적용 하는 건가요 질문3 A현장에서 15일 이상 근무를 지속적으로 하다가 발생 되었다면 출장으로 적용 받을 수 있나요 건설현장에서의 일용근로자는 출장이라는 개념이 성립되지 않는것 같아 여쭤 봅니다 항상 감사합니다 질문1"",2,3 한번에 답변드리겠습니다 A현장과 B현장이 같은 원청밑에 있는 하도급업체의 현장이라면 원도급업체로 산재처리되기 때문에 산재처리는 동일한 원청에서 처리가 될 것이고, A현장과 B현장이 ...",1
180471,"""""뇌출혈 산재처리시 뇌혈관질환 위험요인 -기왕증- 1. 고혈압 심장은 혈압을 내보내는 펌프이고 혈관은 그것을 운반하는 파이프에 해당되는데 혈압이란 혈관 속을 흐르고 있는 혈액이 혈관벽에 미치는 힘이라 할것이며"", 일반적으로 심장이 수축하여 혈액을 내보낼 때 혈관에 가해지는 수축기혈압 또는 최대혈압을 기준으로 고혈압을 구분한다. 구분 저혈압 정상혈압 경계역(고)혈압 고혈압 최고혈압 100이하 140이하 140~160 160이상 최저혈압 60이하 90이하 90~95 95이상 고혈압을 치료하여 정상혈압을 유지해야 하는 이유는 고혈압이 계속되면 혈관에 높은 압력이 생겨 혈관이 약해지고 소동맥류(혹)가 쉽게 생겨 동맥경화를 일으키고 그 결과 혈관이 막히거나 터져 심장질환이나 뇌졸중을 일으키기 때문이다. 2. 비만 비만이란 몸 안에 지방이 과다하게 축적도어 있는 상태를 의미한다...",1
129600,"""""산업재해보상보험법에 따라 산업재해를 당한 경우에는 근로복지공단에 보험급여(요양급여"", 휴업급여, 장해급여, 간병급여, 유족급여, 상병보상연금, 장의비,"" 직업재활급여)를 신청할 수 있으나 동 권리는 3년간 행사하지 않으면 소멸됩니다. 따라서 산업재해를 당한 지 3년이 경과하지 않은 경우라면 사업주의 확인을 받아 산재요양신청을 하거나 사업주가 확인을 거부하는 경우에는 의사의 진단서 등을 첨부하여 근로자가 직접 근로복지공단에 산재요양신청을 하실 수 있습니다."""""",""",1
99466,"""""안녕하십니까.아버지께서 일용직 근로를 하시다가 뇌경색이 와서 치료중이십니다.산재처리를 받을수 있는지"",,,,궁금합니다.치료비가 생각보다 많이 나오네요 2007년10월25일까지 약20여일간 건설회사 잡부일을 하셨습니다올해연세는 64살이시구요 작업내용은 고속국도 터널위의 비탈면에 나무를 심는일을 하셨답니다.그냥 오르내리기도 힘든곳을 나무를 들고올라가서 구덩이를 파고 나무를심고 내려와서는심을 나무를 운반하는 작업을 하셨답니다. 이곳은 대중교통이 불편하여 회사에서 마련해준 숙소에서 모든걸 해결하셨답니다.현장까지 가기위해선 오전 06시에 일어나셔서 세면과 식사를 마치시고 오전 07시부터 작업을 시작하셨고저녁에는 오후6시 이전에는 끝나는일은 거의 없었고 7시 전후로 끝나고 숙소로 돌아오셨다고합니다. 그러시다가 25일 새벽 화장실을 가시려고 일어나는데 자꾸만 몸이 말을 안듣고 한쪽으로 쓰러지셨답니다.그것을 보신분은 옆에 누워 주무시던 동료 아저씨구요. 목격자는 이분 한분 뿐입니...",1
89603,"""""좀 황당한 경우인데 결론적으로 말씀드리면 공상이 아닌 산재처리와 부당해고 두가지 문제로접근하셔야 할 것 같습니다. 1. 산재 문제 회사에서 공상처리를 이행한다면 모르겠지만 이미 퇴직처리를 한 상태라면 공상처리에 대한 의지가 없어 보입니다. 따라서"", 이러한 경우에는 귀하가 산재처리를 통해 보상을 받으시는게 좋을 것으로 보입니다. 산재처리는 회사가 협조하지 않더라도 귀하께서 직접 하시면 되고, 다만 디스크 부분에 대해 업무상 재해를 입증하는데 어려움이 있을 것으로 판단되시면 노무사를 방문하셔서 도움을 부탁하시는게 좋을 것 같습니다. 2. 부당해고 문제 회사와 공상부분에 대해 합의된 상태에서 휴직중 일방적으로 퇴직처리를 하였으므로 부당해고에 해당된다고 보입니다. 다만, 부당해고는 해고된 날로부터 3개월 이내에 하셔야 되는데 퇴직처리가 언제 되었는지 그리고,"" 그러한 해고를 언제 알게 되셨는지가 중요한 문제일 것 같습니다...",1
24834,""""" 사례1 맞벌이를 하고 있는 노동자 A씨"", 18 00경 자가용으로 퇴근하던 중 집 근처 대형마트에 들러 식료품 등을 구입하고 귀가하다가 19 20경 다른 차량과의 접촉사고로 목과 허리를 다친 사고 사례2 평소 출근길에 자녀를 어린이집에 데려다주는 워킹맘 B씨, 09 00경 자녀를 어린이집에 맡기고 출근하던 중 갑자기 차선변경을 하는 옆 차량을 피하다 도로 표지대와 충돌하여 목과 어깨를 다친 사고 사례3 평소 피부병 치료를 받고 있던 노동자 C씨, 18 40경 퇴근 후 한의원에 들러 피부병 치료를 받은 후 귀가하던 중 21 30경 빙판길에 넘어져 좌측 발목이 골절된 사고 고용노동부 장관 김영주 와 근로복지공단 이사장 심

## 3. 모델 및 파일 저장

In [54]:
import pickle
import gzip

# save and compress.
with gzip.open('sanjae_classifier_model.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [55]:
# load and compress.
with gzip.open('H:/S/sanjae/sanjae_classifier_model.pickle', 'rb') as f:
    data = pickle.load(f)
    

In [56]:
daum_view_df.to_csv("./doc_result.csv",encoding="utf-8-sig",index=False)
naver_view_df.to_csv("./doc_result.csv",encoding="utf-8-sig",index=False)

In [93]:
data

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)